# 2022 NHL Playoff Pool
Notebook to help in development of scripts to read brackets and calculate points

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
brackets = os.listdir('brackets')

In [4]:
# def read_brackets_loop(brackets):
#     """Read playoff bracket entries and save results as dict"""
#     for bracket in brackets:
#         df = pd.read_excel("brackets/"+bracket, sheet_name='teams')
#         entrant_name = df['entrant_name'].dropna().item()
#         if entrant_name in picks.keys():
#             entrant_name+=' 2' 
#         picks[entrant_name] = {
#             'rd1_selections' : df['rd1_selections'].dropna().to_list(),
#             'rd2_selections' : df['rd2_selections'].dropna().to_list(),
#             'rd3_selections' : df['rd3_selections'].dropna().to_list(),
#             'rd4_selections' : df['rd4_selections'].dropna().to_list(),
#             'tie_breaker' : df['tie_breaker'].dropna().to_list()
#         }
#     return picks

In [5]:
# read_brackets_loop(brackets)

In [19]:
# Attempt class method to make this cleaner
class bracket_entry:
    """Class to manage bracket entries and calculate points"""
    def __init__(self, bracket_name):
        self.bracket_name = bracket_name
        self.picks = {}
        self.points = {}
        
    def read_brackets(self):
        """Read playoff bracket entries and save results as dict"""
        directory = "brackets/" + self.bracket_name
        df = pd.read_excel(directory, sheet_name='teams')
        self.entrant_name = df['entrant_name'].dropna().item()
        if self.entrant_name in self.picks.keys():
            self.entrant_name = self.entrant_name + ' 2'

        self.picks = {
            'rd1_selections' : df['rd1_selections'].dropna().to_list(),
            'rd2_selections' : df['rd2_selections'].dropna().to_list(),
            'rd3_selections' : df['rd3_selections'].dropna().to_list(),
            'rd4_selections' : df['rd4_selections'].dropna().to_list(),
            'tie_breaker' : df['tie_breaker'].dropna().to_list()
        }
        return
    
    def calc_points(self, results):
        """Calculate pool points based on latest playoff results"""
        points_by_round = {
            'rd1': 2,
            'rd2': 4,
            'rd3': 8,
            'rd4': 17
        }
        for rd in results.keys():
            round_picks = self.picks[rd+'_selections']
            round_result = results[rd]
            round_points = len(set(round_picks) & set(round_result))*points_by_round[rd]
            self.points[rd+'_points'] = round_points
        return

In [8]:
# Initialize entries
entries = {}
for n, bracket in enumerate(brackets):
    print(bracket)
    entries[str(n+1)] = bracket_entry(bracket)

bracket_2022_CameronTest1.xlsx
bracket_2022_CameronTest.xlsx


In [9]:
entries['1'].bracket_name

'bracket_2022_CameronTest1.xlsx'

In [10]:
entries['1'].picks

{}

In [11]:
entries['1'].points

{}

In [12]:
# Extract picks
all_picks = {}
for entry_num, entry in entries.items():
    bracket_entry.read_brackets(entry)
    all_picks[str(entry.entrant_name) + ' ' + str(entry_num)] = entry.picks

In [26]:
pd.DataFrame(all_picks).T

,rd1_selections,rd2_selections,rd3_selections,rd4_selections,tie_breaker
Cameron Harris 1,"[NSH, EDM, COL, MIN, FLA, TBL, CAR, NYR]","[EDM, MIN, FLA, CAR]","[EDM, FLA]",[FLA],[Pasternak]
Cameron Harris 2,"[CGY, EDM, COL, MIN, FLA, TOR, BOS, NYR]","[CGY, COL, FLA, BOS]","[COL, BOS]",[COL],[Pasternak]


In [14]:
entries['1'].picks

{'rd1_selections': ['NSH', 'EDM', 'COL', 'MIN', 'FLA', 'TBL', 'CAR', 'NYR'],
 'rd2_selections': ['EDM', 'MIN', 'FLA', 'CAR'],
 'rd3_selections': ['EDM', 'FLA'],
 'rd4_selections': ['FLA'],
 'tie_breaker': ['Pasternak']}

In [15]:
entries['2'].picks

{'rd1_selections': ['CGY', 'EDM', 'COL', 'MIN', 'FLA', 'TOR', 'BOS', 'NYR'],
 'rd2_selections': ['CGY', 'COL', 'FLA', 'BOS'],
 'rd3_selections': ['COL', 'BOS'],
 'rd4_selections': ['COL'],
 'tie_breaker': ['Pasternak']}

In [16]:
# Results by round
results = {
    'rd1': [
        'CGY',
        'EDM',
        'COL',
        'MIN',
        'FLA',
        'TOR',
        'CAR',
        'PIT'
    ],
    'rd2': [
        'EDM',
        'COL',
        'FLA',
        'PIT'
    ],
    'rd3': [
        'COL',
        'FLA'
    ],
    'rd4': [
        'FLA'
    ]
}

In [17]:
results.keys()

dict_keys(['rd1', 'rd2', 'rd3', 'rd4'])

In [20]:
# Calculate points based on latest results
all_points = {}
for entry_num, entry in entries.items():
    bracket_entry.calc_points(entry, results)
    all_points[str(entry.entrant_name) + ' ' + str(entry_num)] = entry.points

In [23]:
points_df = pd.DataFrame(all_points).T
points_df

,rd1_points,rd2_points,rd3_points,rd4_points
Cameron Harris 1,10,8,8,17
Cameron Harris 2,12,8,8,0


In [24]:
points_df['total_points'] = points_df.loc[:, 'rd1_points':'rd4_points'].sum(axis=1)


In [25]:
points_df

,rd1_points,rd2_points,rd3_points,rd4_points,total_points
Cameron Harris 1,10,8,8,17,43
Cameron Harris 2,12,8,8,0,28
